_Aboa Bouadou, Rose Guionnet, Nathan Laîné, Victor Devys, Tino Margely_

## Import Modules

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import math
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from matplotlib import pyplot as plt
import copy
import time
from pulp import *

In [3]:
SECONDS_PER_DAY = 24*60*60

## Read Data

In [4]:
data = pd.read_excel("data/raw/Orano-données-Single.xlsx", sheet_name="Optim", header=5, usecols=[3,4,5,6,7], nrows=47)

In [5]:
data.head()

,Commandes,ordre,disp,max,durée
0,c1,1,2021-01-01,2021-06-01,6.057056
1,c2,2,2021-02-01,2021-08-01,59.133726
2,c3,3,2021-03-01,2021-10-01,18.738048
3,c4,4,2021-04-01,2021-12-01,6.044729
4,c5,5,2021-05-01,2022-02-01,71.008938


In [6]:
data['disp'].min().to_pydatetime()

datetime.datetime(2021, 1, 1, 0, 0)

# Convert Excel to problem data

In [7]:
def dataframe_to_list(data):
    dataframe = data.copy()
    #dataframe = dataframe
    min_date = dataframe['disp'].min().to_pydatetime()
    dataframe['disp'] = dataframe['disp'].apply(lambda x: (x.to_pydatetime()- min_date).total_seconds()/SECONDS_PER_DAY)
    dataframe['max'] = dataframe['max'].apply(lambda x: (x.to_pydatetime()-min_date).total_seconds()/SECONDS_PER_DAY)
    dataframe['durée'] = dataframe['durée']
    
    return(list(dataframe['disp']),list(dataframe['max']),list(dataframe['durée']))

In [8]:
n = 15
disp, dmax, duree = dataframe_to_list(data)
disp, dmax, duree = disp[:n], dmax[:n], duree[:n]

In [9]:
print("Disp : " , disp[:5], "...")
print("Max : " , dmax[:5], "...")
print("Durée : " , duree[:5], "...")

Disp :  [0.0, 31.0, 59.0, 90.0, 120.0] ...
Max :  [151.0, 212.0, 273.0, 334.0, 396.0] ...
Durée :  [6.057056413817744, 59.13372606760222, 18.738048272122583, 6.044729107183509, 71.0089381707361] ...


In [10]:
def isFeasible(disp, dmax, duree):
    time = sum(duree)
    date_max_max = max(dmax)
    print("time sum : ", time)
    print("Date Max max : ", date_max_max)

In [11]:
isFeasible(disp, dmax, duree)

time sum :  729.4444821614793
Date Max max :  1003.0


## Première résolution

In [12]:
def solvePLNE(disp, dmax, duree):
    
    Tasks= list(range(len(disp))) #Liste des taches
    print("Tasks : ", Tasks)
    M = 100000
    
    model = LpProblem("Orano", LpMaximize)
    
    # Variables 
    x = LpVariable.dicts("x", ((i, j) for i in Tasks for j in Tasks),0,1,LpInteger) #x_{i,j}=1 si i est usinée avant j
    deb = LpVariable.dicts("deb", Tasks,0, None)
    dem = LpVariable.dicts("dem", Tasks,0, None)
    real = LpVariable.dicts("real", Tasks,0, None)
    marge = LpVariable.dicts("marge", Tasks,0, None)
    liv = LpVariable.dicts("liv", Tasks,0, None)
    z = LpVariable("z", lowBound=0)
    
    # Contraintes :
    for j in Tasks:
        model += x[j,j] == 0
        model += dem[j] >= disp[j]
        model += real[j] == dem[j] + duree[j]
        #
        #for i in Tasks:
        #    if i!=j:
        #        model += dem[j] >= real[i] - M*(1-x[i,j]) 
        model += dem[j] >= lpSum(duree[i]*x[i,j] for i in Tasks)
        model += liv[j] == deb[j] + 30
        model += liv[j] <= dmax[j]
        model += marge[j] == dmax[j] - liv[j]
        model += deb[j] >= real[j]
        model += z <= marge[j]
        for i in Tasks:
            if i != j:
                model += x[j,i] + x[i,j] == 1 
            
            for k in Tasks:
                if i!=j and i!=k and j!=k:
                    model += x[i,k] >= x[i,j] + x[j,k] - 1
    
    
    # Objectif : 
    model += z  
    
    #Resolution du PLNE : 
    starttime=time.time() #Pour avoir le temps d'execution
    #model.solve(PULP_CBC_CMD(maxSeconds=15))
    model.solve(PULP_CBC_CMD())
    solveTime=time.time()-starttime
    
    #Affichage des éléments de résolution
    f_val=[]
    for i in range(0,len(Tasks)):
        f_val.append(model.variables()[i].varValue)
    order=np.argsort(f_val)
    
        
    print("Temps de résolution = ", solveTime)
    print("Statut de la solution = ", LpStatus[model.status])
    print("Valeur optimale = ", value(model.objective))
    print("Ordre des taches =",order)
    for i in range(len(Tasks)):
        print(model.variables()[i], model.variables()[i].value())
    for i in range(len(Tasks),2*len(Tasks)):
        print(model.variables()[i], model.variables()[i].value())
    for i in range(2*len(Tasks),3*len(Tasks)):
        print(model.variables()[i], model.variables()[i].value())

    return(solveTime,value(model.objective),list(order))

In [13]:
solvePLNE(disp, dmax, duree)

Tasks :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Temps de résolution =  0.18104076385498047
Statut de la solution =  Optimal
Valeur optimale =  91.866274
Ordre des taches = [ 0  7  1  8 10  9 11 12  6 14 13  3  2  5  4]
deb_0 24.795105
deb_1 90.133726
deb_10 585.24349
deb_11 534.27211
deb_12 729.44448
deb_13 665.8627
deb_14 429.21543
deb_2 77.738048
deb_3 160.9825
deb_4 191.00894
deb_5 180.00649
deb_6 272.39739
deb_7 371.84169
deb_8 463.30558
deb_9 443.31978
dem_0 18.738048
dem_1 31.0
dem_10 534.27211
dem_11 463.30558
dem_12 665.8627
dem_13 585.24349
dem_14 424.0
dem_2 59.0
dem_3 154.93777
dem_4 120.0
dem_5 160.9825
dem_6 181.0
dem_7 271.40388
dem_8 443.31978
dem_9 377.05712
liv_0 54.795105
liv_1 120.13373
liv_10 615.24349
liv_11 564.27211
liv_12 759.44448
liv_13 695.8627
liv_14 459.21543
liv_2 107.73805
liv_3 190.9825
liv_4 221.00894
liv_5 210.00649
liv_6 302.39739
liv_7 401.84169
liv_8 493.30558
liv_9 473.31978


(0.18104076385498047,
 91.866274,
 [0, 7, 1, 8, 10, 9, 11, 12, 6, 14, 13, 3, 2, 5, 4])

[0 1 2 5 3 4 8 6 7 9]

[0 1 2 3 8 4 6 5 7 9]

# Multi Machines

In [14]:
data = pd.read_excel("data/raw/Orano-données-Double.xlsx", sheet_name="Optim", header=5, usecols=[3,4,5,6,7], nrows=47)

In [15]:
n = 15
disp, dmax, duree = dataframe_to_list(data)
disp, dmax, duree = disp[:n], dmax[:n], duree[:n]

In [16]:
print("Disp : " , disp[:5], "...")
print("Max : " , dmax[:5], "...")
print("Durée : " , duree[:5], "...")

Disp :  [0.0, 1.0, 2.0, 3.0, 4.0] ...
Max :  [151.0, 181.0, 212.0, 243.0, 273.0] ...
Durée :  [6.057056413817744, 59.13372606760222, 18.738048272122583, 6.044729107183509, 71.0089381707361] ...


In [17]:
isFeasible(disp, dmax, duree)

time sum :  729.4444821614793
Date Max max :  577.0


In [18]:
def solvePLNEMultiMachines(disp, dmax, duree,M,bigM,mu,a):    
    
    Tasks=range(len(disp)) #Liste des taches 
    Machine=range(M)
    
    model = LpProblem("Orano", LpMaximize)
    
    # Variables 
    y = LpVariable.dicts("y",((i,m) for i in Tasks for m in Machine),0,1,LpInteger) #y_{i}^m si i est usinée sur la machine m
    x = LpVariable.dicts("x", ((i,j,m) for i in Tasks for j in Tasks for m in Machine),0,1,LpInteger) #x_{i,j}^m=1 si i est usinée avant j
    deb = LpVariable.dicts("deb", Tasks,0, None)
    dem = LpVariable.dicts("dem", Tasks,0, None)
    real = LpVariable.dicts("real", Tasks,0, None)
    marge = LpVariable.dicts("marge", Tasks,0, None)
    z3 = LpVariable.dicts("z3", ((i,j,k,m) for i in Tasks for j in Tasks for k in Tasks for m in Machine),0,1,LpInteger)
    z2 = LpVariable.dicts("z2", ((i,j,m) for i in Tasks for j in Tasks for m in Machine),0,1,LpInteger)
    liv = LpVariable.dicts("liv", Tasks,0, None)
    W = LpVariable("W", lowBound=0)
    
    # Contraintes :
    for j in Tasks:
        
        for m in Machine:
            model += x[j,j,m] == 0 
            
        model += lpSum(y[j,m] for m in Machine)==1
        model += dem[j] >= disp[j]
        model += real[j] == dem[j] + duree[j] 
        model += W <= marge[j]
        model += liv[j] == deb[j] + 30
        model += liv[j] <= dmax[j]
        model += marge[j] == dmax[j] - liv[j]
        model += deb[j] >= real[j]
        
        for m in Machine:
            model += dem[j] >= lpSum(duree[i]*x[i,j,m] for i in Tasks)
        
        for i in Tasks:
            if i != j:
                for m in Machine:
                    #model += dem[j]>= real[i]-bigM*(1-x[i,j,m])
                    model += z2[j,i,m] <= y[i,m]
                    model += z2[j,i,m] <= y[j,m]
                    model += z2[j,i,m] >= (y[i,m]+y[j,m])-1
                    model += z2[j,i,m] >= 0
                    model += x[i,j,m]+x[j,i,m]==z2[i,j,m]
                    model += x[j,i,m]<=z2[i,j,m]
                
            for k in Tasks:
                if i!=j and i!=k and j!=k:
                    for m in Machine : 
                        model += z3[i,j,k,m] <= y[i,m]
                        model += z3[i,j,k,m] <= y[j,m]
                        model += z3[i,j,k,m] <= y[k,m]
                        model += z3[i,j,k,m] >= (y[i,m]+y[j,m]+y[k,m])-2
                        model += z3[i,j,k,m] >= 0
                        model += x[i,j,m]+x[j,k,m]-1 <= x[i,k,m]+bigM*(1-z3[i,j,k,m])
                
    # Objectif : 
    model+= W 
    #+0.5*lpSum(mu[i]*a[m]*y[i,m] for i in Tasks for m in Machine)
    
    #Resolution du PLNE : 
    starttime=time.time() #Pour avoir le temps d'execution
    model.solve(PULP_CBC_CMD(maxSeconds=60))
    solveTime=time.time()-starttime
    
    #Affichage des éléments de résolution
    f_val=[]
    for i in range(0,len(Tasks)):
        f_val.append(model.variables()[i].varValue)
    order=np.argsort(f_val)
    
        
    print("Temps de résolution = ", solveTime)
    print("Statut de la solution = ", LpStatus[model.status])
    print("Valeur optimale = ", value(model.objective))
    print("Ordre des taches =",order)
    print("####################")
    for i in range(len(Tasks)+1):
        print(model.variables()[i], model.variables()[i].value())
    print("####################")
    for i in range(len(Tasks)+1,2*len(Tasks)+1):
        print(model.variables()[i], model.variables()[i].value())
    print("####################")
    for i in range(2*len(Tasks)+1,3*len(Tasks)+1):
        print(model.variables()[i], model.variables()[i].value())
    print("####################")
    for i in range(3*len(Tasks)+1,4*len(Tasks)+1):
        print(model.variables()[i], model.variables()[i].value())
    print("####################")
    for i in range(4*len(Tasks)+1,5*len(Tasks)+1):
        print(model.variables()[i], model.variables()[i].value())
    print("####################")
    for i in range(13*len(Tasks)+1,15*len(Tasks)+1):
        print(model.variables()[i], model.variables()[i].value())
    return(solveTime,value(model.objective),list(order))

In [19]:
a=[12,12]
mu=np.random.uniform(0.1,0.7,15)

In [20]:
solvePLNEMultiMachines(disp, dmax, duree,2,10**6,mu,a)

Temps de résolution =  24.382395029067993
Statut de la solution =  Optimal
Valeur optimale =  90.866274
Ordre des taches = [ 8  1 11  2  7  9  0 12 10 14 13  3  6  4  5]
####################
W 90.866274
deb_0 24.795105
deb_1 60.133726
deb_10 286.62567
deb_11 357.59221
deb_12 371.85227
deb_13 308.27049
deb_14 64.349157
deb_2 20.738048
deb_3 70.393886
deb_4 141.40282
deb_5 43.819098
deb_6 135.21649
deb_7 235.65429
deb_8 161.38862
deb_9 227.65128
####################
dem_0 18.738048
dem_1 1.0
dem_10 235.65429
dem_11 286.62567
dem_12 308.27049
dem_13 227.65128
dem_14 59.133726
dem_2 2.0
dem_3 64.349157
dem_4 70.393886
dem_5 24.795105
dem_6 43.819098
dem_7 135.21649
dem_8 141.40282
dem_9 161.38862
####################
liv_0 54.795105
liv_1 90.133726
liv_10 316.62567
liv_11 387.59221
liv_12 401.85227
liv_13 338.27049
liv_14 94.349157
liv_2 50.738048
liv_3 100.39389
liv_4 171.40282
liv_5 73.819098
liv_6 165.21649
liv_7 265.65429
liv_8 191.38862
liv_9 257.65128
####################
marge_0 96.

(24.382395029067993,
 90.866274,
 [8, 1, 11, 2, 7, 9, 0, 12, 10, 14, 13, 3, 6, 4, 5])